# Глубинное обучение для текстовых данных, ФКН ВШЭ

## Домашнее задание 2: Рекуррентные нейронные сети

### Оценивание и штрафы

Максимально допустимая оценка за работу — __10 (+3) баллов__. Сдавать задание после указанного срока сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн: 5.10.25 23:59__   
__Жесткий дедлайн: 8.10.25 23:59__

### О задании

В этом задании вам предстоит самостоятельно реализовать модель LSTM для решения задачи классификации с пересекающимися классами (multi-label classification). Это вид классификации, в которой каждый объект может относиться одновременно к нескольким классам. Такая задача часто возникает при классификации фильмов по жанрам, научных или новостных статей по темам, музыкальных композиций по инструментам и так далее.

В нашем случае мы будем работать с датасетом биотехнических новостей и классифицировать их по темам. Этот датасет уже предобработан: текст приведен к нижнему регистру, удалена пунктуация, все слова разделены проблелом.

In [1]:
import pandas as pd

dataset = pd.read_csv('biotech_news.tsv', sep='\t')
dataset.head()

,text,labels
0,drive your plow over the bones of the dead by ...,other
1,in the recently tabled national budget denel h...,other
2,shares take a break its good for you picture g...,other
3,reso is currently hiring for two positions pro...,other
4,charter buyer club what is the charter buyer c...,other


In [2]:
len(dataset)

3039

## Предобработка лейблов


__Задание 1 (0.5 балла)__. Как вы можете заметить, лейблы записаны в виде строк, разделенных запятыми. Для работы с ними нам нужно преобразовать их в числа. Так как каждый объект может принадлежать нескольким классам, закодируйте лейблы в виде векторов из 0 и 1, где 1 означает, что объект принадлежит соответствующему классу, а 0 – не принадлежит. Имея такую кодировку, мы сможем обучить модель, решая задачу бинарной классификации для каждого класса.

In [3]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

y = mlb.fit_transform(dataset['labels'].str.split(', '))

## Предобработка данных

В этом задании мы будем обучать рекуррентные нейронные сети. Как вы знаете, они работают лучше для коротких текстов, так как не очень хорошо улавливают далекие зависимости. Для уменьшение длин текстов их стоит почистить.

Сразу разделим выборку на обучающую и тестовую, чтобы считать все нужные статистики только по обучающей.

In [4]:
from sklearn.model_selection import train_test_split

texts_train, texts_test, y_train, y_test = train_test_split(
    dataset['text'],
    y,
    test_size=0.2,  # do not change this
    random_state=0  # do not change this
)

__Задание 2 (1 балл)__. Удалите из текстов стоп слова, слишком редкие и слишком частые слова. Гиперпараметры подберите самостоятельно (в идеале их стоит подбирать по качеству на тестовой выборке). Если вы считаете, что стоит добавить еще какую-то обработку, то сделайте это. Важно не удалить ничего, что может повлиять на предсказание класса.

__Задание 3 (1.5 балла)__. Осталось перевести тексты в индексы токенов, чтобы их можно было подавать в модель. У вас есть две опции, как это сделать:
1. __(+0 баллов)__ Токенизировать тексты по словам.
2. __(до +3 баллов)__ Реализовать свою токенизацию BPE. Количество баллов будет варьироваться в зависимости от эффективности реализации. При реализации нельзя пользоваться специализированными библиотеками.

Токенизируйте тексты, переведите их в списки индексов и сложите вместе с лейблами в `DataLoader`. Не забудьте добавить в `DataLoader` `collate_fn`, которая будет дополнять все короткие тексты в батче паддингами. Для маппинга токенов в индексы вам может пригодиться `gensim.corpora.dictionary.Dictionary`.

In [35]:
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from collections import Counter
from gensim.corpora import Dictionary
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from functools import partial

# --- Предобработка ---
STOP_WORDS = list(set(ENGLISH_STOP_WORDS) - {"no", "not"})

def process_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', ' ', text)  # только латиница и пробелы
    tokens = text.split()
    tokens = [w for w in tokens if w not in STOP_WORDS]
    return tokens


train_tokens = [process_text(t) for t in texts_train]
test_tokens  = [process_text(t) for t in texts_test]


df = Counter()
for doc in train_tokens:
    df.update(set(doc))  

min_df = 2
max_df_fraction = 0.9
max_df_count = int(max_df_fraction * len(train_tokens))

valid_words = {w for w, c in df.items() if min_df <= c <= max_df_count}

def filter_tokens(tokens):
    return [t for t in tokens if t in valid_words]

train_tokens_filtered = [filter_tokens(doc) for doc in train_tokens]
test_tokens_filtered  = [filter_tokens(doc) for doc in test_tokens]


vocab = Dictionary(train_tokens_filtered)


special_tokens = ["<pad>", "<unk>", "<bos>", "<eos>"]
for token in special_tokens:
    if token not in vocab.token2id:
        vocab.token2id[token] = len(vocab)
PAD_ID = vocab.token2id["<pad>"]
UNK_ID = vocab.token2id["<unk>"]
BOS_ID = vocab.token2id["<bos>"]
EOS_ID = vocab.token2id["<eos>"]
vocab_size = len(vocab)

# --- Кодировщик с BOS/EOS и max_length ---
def encode(tokens, max_length=None):
    if not tokens:
        return [UNK_ID]

    ids = [BOS_ID] + [vocab.token2id.get(t, UNK_ID) for t in tokens] + [EOS_ID]

    if max_length:
        if len(ids) > max_length:
            ids = ids[:max_length - 1] + [EOS_ID] 
    return ids

MAX_LENGTH = 64

X_train = [encode(t, max_length=MAX_LENGTH) for t in train_tokens_filtered]
X_test  = [encode(t, max_length=MAX_LENGTH) for t in test_tokens_filtered]

# --- Dataset / DataLoader ---
class TextDataset(Dataset):
    def __init__(self, sequences, labels):
        self.seq = sequences
        self.labels = labels

    def __len__(self):
        return len(self.seq)

    def __getitem__(self, i):
        x = torch.tensor(self.seq[i], dtype=torch.long)
        y = torch.tensor(self.labels[i], dtype=torch.float32)
        return x, y

def collate_fn(batch, pad_id):
    seqs, labels = zip(*batch)
    seqs = [torch.tensor(s, dtype=torch.long) for s in seqs]

    padded = pad_sequence(seqs, batch_first=True, padding_value=pad_id)
    labels = torch.stack(labels)
    return padded, labels

collate = partial(collate_fn, pad_id=PAD_ID)

batch_size = 64
train_loader = DataLoader(TextDataset(X_train, y_train), batch_size=batch_size,
                          shuffle=True, collate_fn=collate)
test_loader  = DataLoader(TextDataset(X_test,  y_test),  batch_size=batch_size,
                          shuffle=False, collate_fn=collate)

print(f"Vocab size: {vocab_size}, PAD_ID={PAD_ID}, UNK_ID={UNK_ID}, BOS_ID={BOS_ID}, EOS_ID={EOS_ID}")

Vocab size: 21163, PAD_ID=21159, UNK_ID=21160, BOS_ID=21161, EOS_ID=21162


## Метрика качества

Перед тем, как приступить к обучению, нам нужно выбрать метрику оценки качества. Так как в задаче классификации с пересекающимися классами классы часто несбалансированы, чаще всего в качестве метрики берется [F1 score](https://en.wikipedia.org/wiki/F-score).

Функция `compute_f1` принимает истинные метки и предсказанные и считает среднее значение F1 по всем классам. Используйте ее для оценки качества моделей.

$$
F1_{total} = \frac{1}{K} \sum_{k=1}^K F1(Y_k, \hat{Y}_k),
$$
где $Y_k$ – истинные значения для класса k, а $\hat{Y}_k$ – предсказания.

In [36]:
from sklearn.metrics import f1_score

def compute_f1(y_true, y_pred):
    assert y_true.ndim == 2
    assert y_true.shape == y_pred.shape

    return f1_score(y_true, y_pred, average='macro')

## Обучение моделей

### RNN

В качестве бейзлайна обучим самую простую рекуррентную нейронную сеть. Напомним, что блок RNN выглядит таким образом.

<img src="https://i.postimg.cc/yYbNBm6G/tg-image-1635618906.png" alt="drawing" width="400"/>

Его скрытое состояние обновляется по формуле
$h_t = \sigma(W x_{t} + U h_{t-1} + b_h)$. А предсказание считается с помощью применения линейного слоя к последнему токену
$o_T = V h_T + b_o$. В качестве функции активации выберите гиперболический тангенс. 

__Задание 4 (2 балла)__. Реализуйте RNN в соответствии с формулой выше и обучите ее на нашу задачу. Нулевой скрытый вектор инициализируйте нулями, так модель будет обучаться стабильнее, чем при случайной инициализации. После этого замеряйте качество на тестовой выборке. У вас должно получиться значение F1 не меньше 0.33, а само обучение не должно занимать много времени.

In [37]:
import torch
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_labels):
        super().__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(vocab_size, hidden_size)

        self.input_linear = nn.Linear(hidden_size, hidden_size, bias=False)
        self.hidden_linear = nn.Linear(hidden_size, hidden_size, bias=False)
        self.bias_h = nn.Parameter(torch.zeros(hidden_size))

        self.output_layer = nn.Linear(hidden_size, num_labels)

    def forward(self, input_ids):

        batch_size, seq_len = input_ids.size()
        x_emb = self.embedding(input_ids) 

        h_t = torch.zeros(batch_size, self.hidden_size, device=x_emb.device)


        for x_t in x_emb.transpose(0, 1): 
            h_t = torch.tanh(self.input_linear(x_t) + self.hidden_linear(h_t) + self.bias_h)


        logits = self.output_layer(h_t)
        return logits, h_t


In [70]:
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

hidden_size = 256
num_labels = y_train.shape[1]

model = RNN(vocab_size, hidden_size, num_labels).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=2e-3)

def evaluate(model, loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits, _ = model(x)
            probs = torch.sigmoid(logits)
            preds = (probs > 0.5).int().cpu()
            all_preds.append(preds)
            all_labels.append(y.cpu().int())
    y_true = torch.cat(all_labels).numpy()
    y_pred = torch.cat(all_preds).numpy()
    return compute_f1(y_true, y_pred)

num_epochs = 50
f1_scores = []
for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss = 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits, _ = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    test_f1 = evaluate(model, test_loader)
    f1_scores.append(test_f1)
    print(f"epoch {epoch:02d} | train loss {total_loss/len(train_loader):.4f} | test F1 {test_f1:.4f}")
    


epoch 01 | train loss 0.2166 | test F1 0.0240
epoch 02 | train loss 0.1797 | test F1 0.0090
epoch 03 | train loss 0.1766 | test F1 0.0264
epoch 04 | train loss 0.1661 | test F1 0.0424
epoch 05 | train loss 0.1478 | test F1 0.0470
epoch 06 | train loss 0.1262 | test F1 0.0550
epoch 07 | train loss 0.1054 | test F1 0.0695
epoch 08 | train loss 0.0877 | test F1 0.0980
epoch 09 | train loss 0.0719 | test F1 0.1351
epoch 10 | train loss 0.0590 | test F1 0.1656
epoch 11 | train loss 0.0485 | test F1 0.1895
epoch 12 | train loss 0.0402 | test F1 0.2192
epoch 13 | train loss 0.0339 | test F1 0.2533
epoch 14 | train loss 0.0286 | test F1 0.2571
epoch 15 | train loss 0.0242 | test F1 0.2752
epoch 16 | train loss 0.0214 | test F1 0.3229
epoch 17 | train loss 0.0192 | test F1 0.3087
epoch 18 | train loss 0.0173 | test F1 0.3301
epoch 19 | train loss 0.0158 | test F1 0.3271
epoch 20 | train loss 0.0145 | test F1 0.3217
epoch 21 | train loss 0.0134 | test F1 0.3321
epoch 22 | train loss 0.0126 | tes

In [ ]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params

5556509

In [72]:
max(f1_scores)

0.35040835298100204

### LSTM

<img src="https://i.postimg.cc/pL5LdmpL/tg-image-2290675322.png" alt="drawing" width="400"/>

Теперь перейдем к более продвинутым рекурренным моделям, а именно LSTM. Из-за дополнительного вектора памяти эта модель должна гораздо лучше улавливать далекие зависимости, что должно напрямую отражаться на качестве.

Параметры блока LSTM обновляются вот так ($\sigma$ означает сигмоиду):
\begin{align}
f_{t} &= \sigma(W_f x_{t} + U_f h_{t-1} + b_f) \\ 
i_{t} &= \sigma(W_i x_{t} + U_i h_{t-1} + b_i) \\
\tilde{c}_{t} &= \tanh(W_c x_{t} + U_c h_{t-1} + b_i) \\
c_{t} &= f_t \odot c_{t-1} + i_t \odot \tilde{c}_t \\
o_{t} &= \sigma(W_t x_{t} + U_t h_{t-1} + b_t) \\
h_t &= o_t \odot \tanh(c_t)
\end{align}

__Задание 5 (2 балла).__ Реализуйте LSTM по описанной схеме. Выберите гиперпараметры LSTM так, чтобы их общее число (без учета слоя эмбеддингов) примерно совпадало с числом параметров обычной RNN, но размерность скрытого слоя была не меньше 64. Так мы будем сравнивать архитектуры максимально независимо. Обучите LSTM до сходимости и сравните качество с RNN на тестовой выборке. Удалось ли получить лучший результат? Как вы можете это объяснить?

In [73]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)

        # Все веса: W_* (для x_t) и U_* (для h_{t-1})
        self.W_f = nn.Linear(embedding_dim, hidden_dim)
        self.U_f = nn.Linear(hidden_dim, hidden_dim, bias=True)

        self.W_i = nn.Linear(embedding_dim, hidden_dim)
        self.U_i = nn.Linear(hidden_dim, hidden_dim, bias=True)

        self.W_c = nn.Linear(embedding_dim, hidden_dim)
        self.U_c = nn.Linear(hidden_dim, hidden_dim, bias=True)

        self.W_o = nn.Linear(embedding_dim, hidden_dim)
        self.U_o = nn.Linear(hidden_dim, hidden_dim, bias=True)

        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # x: (batch, seq_len)
        embedded = self.embedding(x)  # (batch, seq_len, emb_dim)
        batch_size, seq_len, _ = embedded.shape

        h_t = torch.zeros(batch_size, self.U_f.out_features, device=x.device)
        c_t = torch.zeros_like(h_t)

        for t in range(seq_len):
            x_t = embedded[:, t, :]

            f_t = torch.sigmoid(self.W_f(x_t) + self.U_f(h_t))
            i_t = torch.sigmoid(self.W_i(x_t) + self.U_i(h_t))
            g_t = torch.tanh(self.W_c(x_t) + self.U_c(h_t))
            c_t = f_t * c_t + i_t * g_t
            o_t = torch.sigmoid(self.W_o(x_t) + self.U_o(h_t))
            h_t = o_t * torch.tanh(c_t)

        logits = self.fc(h_t)
        return logits


In [74]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_labels = y_train.shape[1]

model = MyLSTM(vocab_size, embedding_dim=256, hidden_dim=256,
               output_dim=y_train.shape[1], pad_idx=PAD_ID).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)

def evaluate(model, loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            probs = torch.sigmoid(logits)
            preds = (probs > 0.5).int().cpu()
            all_preds.append(preds)
            all_labels.append(y.cpu().int())
    y_true = torch.cat(all_labels).numpy()
    y_pred = torch.cat(all_preds).numpy()
    return compute_f1(y_true, y_pred)

num_epochs = 50
for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss = 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    test_f1 = evaluate(model, test_loader)
    f1_scores.append(test_f1)
    print(f"epoch {epoch:02d} | train loss {total_loss/len(train_loader):.4f} | test F1 {test_f1:.4f}")


epoch 01 | train loss 0.2365 | test F1 0.0240
epoch 02 | train loss 0.1800 | test F1 0.0240
epoch 03 | train loss 0.1802 | test F1 0.0346
epoch 04 | train loss 0.1800 | test F1 0.0240
epoch 05 | train loss 0.1798 | test F1 0.0240
epoch 06 | train loss 0.1799 | test F1 0.0346
epoch 07 | train loss 0.1801 | test F1 0.0240
epoch 08 | train loss 0.1800 | test F1 0.0084
epoch 09 | train loss 0.1803 | test F1 0.0091
epoch 10 | train loss 0.1798 | test F1 0.0240
epoch 11 | train loss 0.1798 | test F1 0.0240
epoch 12 | train loss 0.1797 | test F1 0.0106
epoch 13 | train loss 0.1798 | test F1 0.0240
epoch 14 | train loss 0.1799 | test F1 0.0240
epoch 15 | train loss 0.1774 | test F1 0.0204
epoch 16 | train loss 0.1715 | test F1 0.0212
epoch 17 | train loss 0.1643 | test F1 0.0224
epoch 18 | train loss 0.1595 | test F1 0.0230
epoch 19 | train loss 0.1552 | test F1 0.0396
epoch 20 | train loss 0.1521 | test F1 0.0421
epoch 21 | train loss 0.1484 | test F1 0.0454
epoch 22 | train loss 0.1447 | tes

In [77]:

sum(p.numel() for p in model.parameters() if p.requires_grad)

5951517

In [78]:
max(f1_scores)

0.36965653399950305

Обучение шло дольше, но зато за тоже количество эпох и параметров удалось получить лучше результат (~ на 0.02). Предполагаю, что при большем количестве эпох можно еще лучще получить f1 скор. Могу объяснить это улучшенной архитектурой LSTM с более умной памятью (использование гейтов и тд)

__Задание 6 (2 балла).__ Главный недостаток RNN моделей заключается в том, что при сжатии всей информации в один вектор, важные детали пропадают. Для решения этой проблемы был придуман механизм внимания. Реализуйте его по [оригинальной статье](https://arxiv.org/abs/1409.0473). Замерьте качество и сделайте выводы.   
Обратите внимание, что метод был предложен для Encoder-Decoder моделей. В нашем случае декодера нет, поэтому встройте внимание в энкодер: каждый блок LSTM будет смотреть на выходы всех предыдущих блоков.   

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AttentionLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.hidden_dim = hidden_dim


        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)


        self.W_h = nn.Linear(hidden_dim, hidden_dim)
        self.W_s = nn.Linear(hidden_dim, hidden_dim)
        self.v_a = nn.Linear(hidden_dim, 1, bias=False)
        self.W_c = nn.Linear(hidden_dim * 2, hidden_dim)

        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)  # (B, T, E)
        h_seq, (h_T, c_T) = self.lstm(embedded)  # h_seq: (B, T, H)


        enhanced_h = []

        for t in range(h_seq.size(1)):
            h_t = h_seq[:, t, :]  # (B, H)

            past_h = h_seq[:, :t+1, :]  # (B, t+1, H)


            scores = self.v_a(torch.tanh(
                self.W_h(past_h) + self.W_s(h_t).unsqueeze(1)
            )).squeeze(-1)  # (B, t+1)


            attn_weights = F.softmax(scores, dim=-1)  # (B, t+1)


            c_t = torch.bmm(attn_weights.unsqueeze(1), past_h).squeeze(1)  # (B, H)

            h_t_tilde = torch.tanh(self.W_c(torch.cat([h_t, c_t], dim=-1)))  # (B, H)
            enhanced_h.append(h_t_tilde.unsqueeze(1))


        enhanced_h = torch.cat(enhanced_h, dim=1)  # (B, T, H)


        logits = self.fc(enhanced_h[:, -1, :])
        return logits


In [80]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_labels = y_train.shape[1]

model = AttentionLSTM(vocab_size, embedding_dim=256, hidden_dim=256,
               output_dim=y_train.shape[1], pad_idx=PAD_ID).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)

def evaluate(model, loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            probs = torch.sigmoid(logits)
            preds = (probs > 0.5).int().cpu()
            all_preds.append(preds)
            all_labels.append(y.cpu().int())
    y_true = torch.cat(all_labels).numpy()
    y_pred = torch.cat(all_preds).numpy()
    return compute_f1(y_true, y_pred)

num_epochs = 50
for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss = 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    test_f1 = evaluate(model, test_loader)
    f1_scores.append(test_f1)
    print(f"epoch {epoch:02d} | train loss {total_loss/len(train_loader):.4f} | test F1 {test_f1:.4f}")


epoch 01 | train loss 0.2309 | test F1 0.0239
epoch 02 | train loss 0.1815 | test F1 0.0000
epoch 03 | train loss 0.1805 | test F1 0.0240
epoch 04 | train loss 0.1810 | test F1 0.0240
epoch 05 | train loss 0.1808 | test F1 0.0240
epoch 06 | train loss 0.1815 | test F1 0.0240
epoch 07 | train loss 0.1806 | test F1 0.0240
epoch 08 | train loss 0.1806 | test F1 0.0000
epoch 09 | train loss 0.1811 | test F1 0.0240
epoch 10 | train loss 0.1802 | test F1 0.0099
epoch 11 | train loss 0.1811 | test F1 0.0000
epoch 12 | train loss 0.1806 | test F1 0.0240
epoch 13 | train loss 0.1810 | test F1 0.0240
epoch 14 | train loss 0.1814 | test F1 0.0240
epoch 15 | train loss 0.1811 | test F1 0.0240
epoch 16 | train loss 0.1809 | test F1 0.0338
epoch 17 | train loss 0.1811 | test F1 0.0099
epoch 18 | train loss 0.1808 | test F1 0.0240
epoch 19 | train loss 0.1815 | test F1 0.0240
epoch 20 | train loss 0.1806 | test F1 0.0240
epoch 21 | train loss 0.1808 | test F1 0.0000
epoch 22 | train loss 0.1808 | tes

In [81]:
sum(p.numel() for p in model.parameters() if p.requires_grad)


6214685

In [82]:
max(f1_scores)


0.4137022086923262

Качество еще улучишлось :) СЮДАААА

__Задание 7 (1 балл).__ Добавьте в вашу реализации возможность увеличивать число слоев LSTM. Обучите модель с двумя слоями и замерьте качество. Сделайте выводы: стоит ли увеличивать размер модели?

In [83]:
class StackedAttentionLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, pad_idx, num_layers=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
        )


        self.W_h = nn.Linear(hidden_dim, hidden_dim)
        self.W_s = nn.Linear(hidden_dim, hidden_dim)
        self.v_a = nn.Linear(hidden_dim, 1, bias=False)
        self.W_c = nn.Linear(hidden_dim * 2, hidden_dim)

        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        h_seq, (h_n, c_n) = self.lstm(embedded)


        enhanced_h = []
        for t in range(h_seq.size(1)):
            h_t = h_seq[:, t, :]
            past_h = h_seq[:, :t+1, :]

            scores = self.v_a(torch.tanh(
                self.W_h(past_h) + self.W_s(h_t).unsqueeze(1)
            )).squeeze(-1)

            attn_weights = torch.softmax(scores, dim=-1)
            c_t = torch.bmm(attn_weights.unsqueeze(1), past_h).squeeze(1)
            h_t_tilde = torch.tanh(self.W_c(torch.cat([h_t, c_t], dim=-1)))
            enhanced_h.append(h_t_tilde.unsqueeze(1))

        enhanced_h = torch.cat(enhanced_h, dim=1)
        logits = self.fc(enhanced_h[:, -1, :])
        return logits


In [85]:
model = StackedAttentionLSTM(
    vocab_size=vocab_size,
    embedding_dim=100,
    hidden_dim=64,
    output_dim=y_train.shape[1],
    pad_idx=PAD_ID,
    num_layers=2
).to(device)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_labels = y_train.shape[1]

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)

def evaluate(model, loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            probs = torch.sigmoid(logits)
            preds = (probs > 0.5).int().cpu()
            all_preds.append(preds)
            all_labels.append(y.cpu().int())
    y_true = torch.cat(all_labels).numpy()
    y_pred = torch.cat(all_preds).numpy()
    return compute_f1(y_true, y_pred)

num_epochs = 50
for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss = 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    test_f1 = evaluate(model, test_loader)
    f1_scores.append(test_f1)
    print(f"epoch {epoch:02d} | train loss {total_loss/len(train_loader):.4f} | test F1 {test_f1:.4f}")


epoch 01 | train loss 0.3214 | test F1 0.0000
epoch 02 | train loss 0.1811 | test F1 0.0230
epoch 03 | train loss 0.1803 | test F1 0.0230
epoch 04 | train loss 0.1804 | test F1 0.0230
epoch 05 | train loss 0.1803 | test F1 0.0230
epoch 06 | train loss 0.1802 | test F1 0.0230
epoch 07 | train loss 0.1803 | test F1 0.0230
epoch 08 | train loss 0.1804 | test F1 0.0230
epoch 09 | train loss 0.1804 | test F1 0.0230
epoch 10 | train loss 0.1805 | test F1 0.0000
epoch 11 | train loss 0.1803 | test F1 0.0230
epoch 12 | train loss 0.1806 | test F1 0.0000
epoch 13 | train loss 0.1805 | test F1 0.0230
epoch 14 | train loss 0.1803 | test F1 0.0230
epoch 15 | train loss 0.1807 | test F1 0.0230
epoch 16 | train loss 0.1804 | test F1 0.0230
epoch 17 | train loss 0.1805 | test F1 0.0230
epoch 18 | train loss 0.1803 | test F1 0.0230
epoch 19 | train loss 0.1803 | test F1 0.0230
epoch 20 | train loss 0.1803 | test F1 0.0197
epoch 21 | train loss 0.1794 | test F1 0.0199
epoch 22 | train loss 0.1757 | tes

Качество ухудшилось :( Причем существенно - могу сделать вывод, что не следует делать увеличение количества слоев (иначе придется тюнить гиперпараметры, что возможно усложнит задачу) 